This web application provides a way to explore ECMWF open datasets available at their [public portal](https://github.com/ecmwf/ecmwf-opendata) https://github.com/ecmwf/ecmwf-opendata. 

ECMWF open forecast dataset is available on their [portal https://data.ecmwf.int/forecasts](https://data.ecmwf.int/forecasts)

The application has interactive input fields that can be used to define data parameters and forecast date.  


In [ ]:
# Application initialization
import mercury as mr

show_code = mr.Checkbox(value=False, label="Show code")

app = mr.App(
    title='Interactive IFS Data Exploration',
    description='Loading and Visualizing IFS Open Data',
    show_code=show_code.value
    
)


In [ ]:
# helper functions
from cgan_ui.show_forecasts import get_possible_variables

get_possible_variables()

In [ ]:
from cgan_ui.constants import AOI_BBOX

mask_values = list(AOI_BBOX.keys())
mask = mr.Select(
    choices=mask_values,
    value=mask_values[0],
    label="Area of Interest",
    url_key="mask_area",
)

In [ ]:
from cgan_ui.utils import get_forecast_data_dates

data_dates = get_forecast_data_dates(mask=mask.value)
data_dates

In [ ]:
ifs_date = mr.Select(
    choices=data_dates,
    value=data_dates[-1],
    label="Forecast Initialization Date",
    url_key="ifs_date",
)

In [ ]:
from cgan_ui.constants import DATA_PARAMS

params = list(DATA_PARAMS.keys())

vis_param = mr.Select(
    value=params[0], choices=params, label="Data Variable to Plot", url_key="vis_param"
)

In [ ]:
# helper functions
from datetime import datetime
from cgan_ui.utils import get_data_store_path
from cgan_ui.ecmwf_esm_viz import load_forecast

source = "ecmwf"
stream = "enfo"

store_path = get_data_store_path()
data_dir = store_path / "interim" / mask.value / source / stream
init_date = datetime.strptime(ifs_date.value, "%b %d, %Y")

# load datasets
data = load_forecast(
    key=vis_param.value, forecast_init_date=init_date, data_dir=data_dir
)

In [ ]:
from cgan_ui.ecmwf_esm_viz import plot_forecast

# Plot the current data
plot_forecast(data)

In [ ]:
import shutil
import tarfile
from loguru import logger
from pathlib import Path
from cgan_ui.constants import LEAD_START_HOUR, LEAD_END_HOUR

download = mr.Button(label="Download Data")

# prepare data downloads on button click
if download.clicked:
    # output_dir will be current directory when running in the Jupyter Notebook
    # output_dir will be some temporary directory path when running in Mercury
    output_dir = mr.OutputDir()
    d = init_date
    file_ext = "nc"
    forecast_hours = (
        [LEAD_START_HOUR, LEAD_END_HOUR]
        if DATA_PARAMS[vis_param.value]["accumulated"]
        else range(LEAD_START_HOUR, LEAD_END_HOUR + 1, 3)
    )
    # Just need the start and end lead times for accumulated variables
    for lead_hour in forecast_hours:
        # Name of the file we will read
        file_name = (
            f"{d.year}{d.month:02}{d.day:02}000000-{lead_hour}h-enfo-ef.{file_ext}"
        )
        # copy files to output dir
        ds_path = data_dir / file_name
        output_file = Path(output_dir.path) / f"{mask.value.lower()}-{file_name}"
        try:
            shutil.copy(ds_path, output_file)
        except Exception as err:
            logger.error(f"failed to copy {file_name} to output directory")
